# 数据库操作

## 登录方式： sock登录
#需要先在localhost才可以使用此种方式登录，需要直接调用sock文件

[root@localhost agr]# mysql -S /mysql/3307/temp/mysql_3307.sock 

Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 4
Server version: 5.6.50-log MySQL Community Server (GPL)

Copyright (c) 2000, 2020, Oracle and/or its affiliates. All rights reserved.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

mysql> 


#一般需要加入 -p 参数输入密码
[root@localhost agr]# mysql -p -S  /mysql/3307/temp/mysql_3307.sock 


## 添加用户及修改密码

mysql> show databases;
+--------------------+
| Database        |
+--------------------+
| information_schema |
| mysql          |
| performance_schema |
| test           |
+--------------------+
4 rows in set (0.00 sec)

mysql> use mysql;
Database changed
mysql> select user,host,password from user;
+------+-----------------------+----------+
| user | host             | password |
+------+-----------------------+----------+
| root | localhost         |          |
| root | localhost.localdomain |          |
| root | 127.0.0.1         |          |
| root | ::1             |          |
|      | localhost        |          |
|      | localhost.localdomain |          |
+------+-----------------------+----------+
6 rows in set (0.00 sec)

mysql> 



### 修改密码的方式

#### 方式一 用SET PASSWORD命令

格式：mysql> set password for 用户名@‘localhost’ = password('新密码');  

例子：mysql> set password for root@‘localhost’ = password('1234');  

#### 方式二 用mysqladmin

格式：mysqladmin -u用户名 -p旧密码 password 新密码  

[root@localhost agr]# mysqladmin -uroot -p -P 3307 password 1234
Enter password: 

#### 方式三 用UPDATE 直接编辑表user

mysql> use mysql;  

mysql> update user set password=password('123') where user='root' and host='localhost';  

mysql> flush privileges;  




## 忘记密码的越过密码登录的操作

### 检查mysql服务是否启动，如果启动，关闭mysql服务

#检查是否已经启动
[root@localhost agr]# netstat -tunlp|grep mysql

    tcp6       0      0 :::3307                 :::*                 LISTEN      4189/mysqld         
    tcp6       0      0 :::3308                 :::*                 LISTEN      4569/mysqld         

#关闭服务
[root@localhost agr]# /mysql/3307/mysql_3307 stop

### 修改my.cnf文件
在/etc/my.cnf， 中插入以下两行代码
[mysqld]
skip-grant-tables

[root@localhost agr]# vim /etc/my.cnf

[mysqld]
skip-grant-tables

### 免密码登录数据库并采用上一步中的操作修改密码




# 创建数据库，插入数据表

## 创建数据库格式
CREATE {DATABASE | SCHEMA} [IF NOT EXISTS] db_name
    [create_option] ...
create_option: [DEFAULT] {
    CHARACTER SET [=] charset_name
  | COLLATE [=] collation_name
}

mysql> CREATE DATABASE IF NOT EXISTS lol 
    -> DEFAULT CHARACTER SET utf8
    -> DEFAULT COLLATE utf8_general_ci; #有些版本是utf8_chinese_ci;

###  验证
mysql> show create database lol;
+----------+--------------------------------------------------------------+
| Database | Create Database                                |
+----------+--------------------------------------------------------------+
| lol      | CREATE DATABASE `lol` /*!40100 DEFAULT CHARACTER SET utf8 */ |
+----------+--------------------------------------------------------------+
1 row in set (0.00 sec)



## 插入数据表

mysql> create table if not exists tanks(
    -> id int, 
    -> name  varchar(100) not null, 
    -> skills varchar(255) not null, 
    -> price int not null, 
    -> primary key(id)  
    -> ) engine=InnoDB DEFAULT CHARSET = utf8;
Query OK, 0 rows affected (0.01 sec)

### 验证

mysql> show create table tanks;
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Table | Create Table                                                                                                                                                                                                        |
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| tanks | CREATE TABLE `tanks` (
  `id` int(11) NOT NULL DEFAULT '0',
  `name` varchar(100) NOT NULL,
  `skills` varchar(255) NOT NULL,
  `price` int(11) NOT NULL,
  PRIMARY KEY (`id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8 |
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+


mysql> desc tanks;
+--------+--------------+------+-----+---------+-------+
| Field  | Type      | Null | Key | Default | Extra |
+--------+--------------+------+-----+---------+-------+
| id    | int(11)     | NO   | PRI   | 0     |       |
| name   | varchar(100) | NO   |     | NULL    |       |
| skills | varchar(255)  | NO   |     | NULL    |       |
| price  | int(11)     | NO   |     | NULL    |       |
+--------+--------------+------+-----+---------+-------+
4 rows in set (0.01 sec)




## 往数据标中写入数据


mysql> insert into tanks(id,name,skills,price) values(1,'程咬金','大招回血，血量越低，伤害越高'，8888）,(2,'庄周','大招解除控制，为队友添加护盾',2888);
### 验证
mysql> select * from tanks;
+----+-----------+------------------------------------------+-------+
| id | name      | skills                      | price |
+----+-----------+------------------------------------------+-------+
|  1 | 程咬金    | 大招回血,血量越低,伤害越高          |  8888 |
|  2 | 庄周      | 大招解除控制,为队友添加护盾        |  2888 |
+----+-----------+------------------------------------------+-------+
2 rows in set (0.00 sec)


## 修改数据表

## 增

###  添加数据表字段

mysql> alter table [表名] add [字段名] [字段数据类型（长度）] 额外的字段属性；

### 添加指定位置的字段

mysql> alter table [表名] add [字段名] after [字段名]；

### 添加多个字段

mysql> alter table [表名] add [字段名1、类型1] ， add [字段名2、类型2] ；

#特殊例子，指定位置插入多个字段
mysql> alter table [表名] add [字段名1、类型1] after [字段a]， add [字段名2、类型2] after [字段b]； 

## 删

### 删除数据表

drop table [表名]； 

### 删除table中的字段

mysql> alter table [表名] drop [字段名]；

### 删除数据表的内容

#### 方式一 delete语句
    mysql> delete from [数据表]；#由于delete会保留id(key)序号，清空语句执行后，id(key)自增属性丢失

#### id(key)自增属性丢失解决方法：添加自增属性
    #modify语句，可以修改字段数据类型、额外属性（在此用到的功能二）

    mysql> alter table tanks modify id int(11) auto_increment; #auto_increment id自增


#### 带条件删除表中指定内容

    #delete是一行一行地删除，对大容量表删除很慢。但是delete会保留序号
    举个例子，删除含（1、2、3）的所有内容后，再次插入数据将从4开始（已添加增属性）
    #用where语句判断

    mysql> delete from [表名] where [条件a?b]；

      #delete from tanks where id=3;   删了3，之后id=4的不会变成3

#### 方式二 tuncate语句
#原理：重新创建一个新的数据表，完全抛弃原有内容

mysql>truncket table [表名];


## 改

### 修改数据表名字

#### 方式一 rename

mysql> rename table [旧名字] to [新名字]；

#### 方式二 alter

mysql> alter table [旧的表名] rename to [新名字]；

### 修改表字段的数据类型

mysql> alter table [表名] change [旧字段名] [新字段名] [新数据类型] ；

### 改表中数据 update语句

mysql> update [数据库].[表名] set [修改字段=b] where [定位条件a?b];#若无精准定位条件，将会把其他数据行也修改

mysql> update [数据库].[表名] set [修改字段1=b]，[修改字段2=c] where [定位条件a?b];#同时改多个字段

## 查

### 查询表的全部内容 ，用 *

mysql> select * from [表名]；

### 查询表的某些字段 ，填入字段名

mysql> select [字段名]，[字段名] from [表名]；

### 查询符合条件的：where 语句

mysql> select [字段名]，[字段名] from [表名] where [条件a?b];

mysql> select [字段名]，[字段名] from [表名] where [条件a?b] and [条件c?d];

### 限制结果查询的数量: limit 语句

mysql> select [字段名]，[字段名] from [表名] where [条件a?b] limit [起始位置][数量]；
#不包含起始位置




## 索引

### 查看当前表的索引
mysql> show index from tanks \G；
+-------+------------+----------+--------------+-------------+-----------+-------------+----------+--------+------+------------+---------+---------------+
| Table | Non_unique | Key_name | Seq_in_index | Column_name | Collation | Cardinality | Sub_part | Packed | Null | Index_type | Comment | Index_comment |
+-------+------------+----------+--------------+-------------+-----------+-------------+----------+--------+------+------------+---------+---------------+
| tanks |          0 | PRIMARY  |            1 | id          | A         |           2 |     NULL | NULL   |      | BTREE      |         |               |
+-------+------------+----------+--------------+-------------+-----------+-------------+----------+--------+------+------------+---------+---------------+
1 row in set (0.00 sec)

### 添加索引
mysql> alter table [表名] add index [要设置的索引名（字段名）]；

mysql> alter table tanks add index name_index(name);
Query OK, 0 rows affected (0.02 sec)
Records: 0  Duplicates: 0  Warnings: 0

#### 检查
mysql> show index from tanks \G;
*************************** 1. row ***************************
        Table: tanks
   Non_unique: 0
     Key_name: PRIMARY
 Seq_in_index: 1
  Column_name: id
    Collation: A
  Cardinality: 2
     Sub_part: NULL
       Packed: NULL
         Null: 
   Index_type: BTREE
      Comment: 
Index_comment: 
*************************** 2. row ***************************
        Table: tanks
   Non_unique: 1
     Key_name: name_index
 Seq_in_index: 1
  Column_name: name
    Collation: A
  Cardinality: 2
     Sub_part: NULL
       Packed: NULL
         Null: 
   Index_type: BTREE
      Comment: 
Index_comment: 
2 rows in set (0.00 sec)

### 删除索引

alter table [表名] drop index [索引名]；





# 排序

## 查询结果排序： order by语句

mysql> select [字段名1]，[字段名2]，[字段名3] from [表名] order by [字段名2]；#由按默认的字段（假设是1）排序改成按字段2排序

#### 排序默认是升序asc,改成降序desc

mysql> select [字段名1]，[字段名2]，[字段名3] from [表名] order by [字段名2] desc；